In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from load_data import *

train_datas, train_labels=load_train_data()
x_train, x_valid, y_train, y_valid = train_test_split(train_datas, train_labels, test_size=0.1)

#每个批次的大小
batch_size = 32

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    #x input tensor of shape `[batch, in_height, in_width, in_channels]`
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,96,96,3])#96*96
y = tf.placeholder(tf.float32,[None,2])


#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([3,3,3,32])#3*3的采样窗口，3个卷积核从3个平面抽取特征
b_conv1 = bias_variable([32])#每一个卷积核一个偏置值

#把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)#进行max-pooling

#初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([3,3,32,64])#3*3的采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([64])#每一个卷积核一个偏置值

#把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)#进行max-pooling

#初始化第三个卷积层的权值和偏置
W_conv3 = weight_variable([3,3,64,128])#5*5的采样窗口，128个卷积核从64个平面抽取特征
b_conv3 = bias_variable([128])#每一个卷积核一个偏置值

#把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)#进行max-pooling

#初始化第四个卷积层的权值和偏置
W_conv4 = weight_variable([3,3,128,256])#5*5的采样窗口，256个卷积核从128个平面抽取特征
b_conv4 = bias_variable([256])#每一个卷积核一个偏置值

#把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv4 = tf.nn.relu(conv2d(h_pool3,W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)#进行max-pooling

#初始化第五个卷积层的权值和偏置
W_conv5 = weight_variable([3,3,256,256])#3*3的采样窗口，256个卷积核从256个平面抽取特征
b_conv5 = bias_variable([256])#每一个卷积核一个偏置值

#把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv5 = tf.nn.relu(conv2d(h_pool4,W_conv5) + b_conv5)
h_pool5 = max_pool_2x2(h_conv5)#进行max-pooling

#初始化第一个全连接层的权值
W_fc1 = weight_variable([3*3*256,1024])#上一层有3*3*256个神经元，全连接层有1024个神经元
b_fc1 = bias_variable([1024])#1024个节点

#把池化层5的输出扁平化为1维
h_pool5_flat = tf.reshape(h_pool5,[-1,3*3*256])
#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool5_flat,W_fc1) + b_fc1)

#keep_prob用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层
W_fc2 = weight_variable([1024,2])
b_fc2 = bias_variable([2])

#计算输出
y_=tf.matmul(h_fc1_drop,W_fc2) + b_fc2
prediction = tf.nn.softmax(y_)

#交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))#argmax返回一维张量中最大的值所在的位置

#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    is_train=False
    saver=tf.train.Saver(max_to_keep=1)
    if is_train:
        for n in range(len(x_train) //batch_size +1):
            batch_xs = x_train[n*batch_size:n*batch_size+batch_size]
            batch_ys = y_train[n*batch_size:n*batch_size+batch_size]
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.5})
            if n % 100==0:
                loss=sess.run(cross_entropy,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
                Accuracy=sess.run(accuracy,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
                val_loss=sess.run(cross_entropy,feed_dict={x:x_valid,y:y_valid,keep_prob:1.0})
                val_Accuracy=sess.run(accuracy,feed_dict={x:x_valid,y:y_valid,keep_prob:1.0})
                print("batch "+str(n)+" ,train_loss "+\
                          "{:.5f}".format(loss)+" ,training accuracy "+str(Accuracy))
                print("batch "+str(n)+" ,val_loss "+\
                          "{:.5f}".format(val_loss)+" ,val_Accuracy "+str(val_Accuracy))
                saver.save(sess,'ckpt/latest.ckpt')
    
    
    else:
        model_file=tf.train.latest_checkpoint('ckpt/')
        saver.restore(sess,model_file)
        test_datas, test_labels=load_test_data()
        average_accuracy=[]
        batch_size=1000
        for n in range(len(test_datas) //batch_size ):
            batch_xs = test_datas[n*batch_size:n*batch_size+batch_size]
            batch_ys = test_labels[n*batch_size:n*batch_size+batch_size]      
            Accuracy=sess.run(accuracy,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
            average_accuracy.append(Accuracy)
            print("batch "+str(n)+" ,test_accuracy "+str(Accuracy))    
        average_accuracy= sum(average_accuracy)/len(average_accuracy)
        print("average_accuracy "+str(average_accuracy))

/Users/yangbinfeng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


INFO:tensorflow:Restoring parameters from ckpt/latest.ckpt
batch 0 ,test_accuracy 0.991
batch 1 ,test_accuracy 0.988
batch 2 ,test_accuracy 0.989
batch 3 ,test_accuracy 0.993
batch 4 ,test_accuracy 0.989
average_accuracy 0.989999997616
